# Plan and Execute Pattern using Semantic Kernel

This notebook demonstrates the implementation of the Plan and Execute pattern using Semantic Kernel. This pattern improves agent performance by:

1. Breaking down complex tasks into manageable sub-tasks (Planning)
2. Executing each sub-task in sequence
3. Adapting to feedback during execution

## Architecture Overview

The Plan and Execute pattern involves:
- **Planner**: Responsible for generating a structured plan of sub-tasks
- **Executor**: Handles the execution of each sub-task
- **Memory**: Maintains context between steps
- **Tools**: Custom functions that can be called during execution

![Plan and Execute Pattern](../../../1_agentic-design-ptn/images/planning.png)

## Understanding the Plan and Execute Pattern

The Plan and Execute pattern we've demonstrated offers several advantages:

1. **Task Decomposition**: Complex tasks are broken down into simpler, manageable steps
2. **Tool Selection**: The planner automatically selects the appropriate tools for each step
3. **Adaptability**: If a step fails, the planner can adapt by trying alternative approaches
4. **Explainability**: The plan provides transparency into how the agent approaches problems

This pattern is particularly useful for tasks that require multiple steps or the use of various tools to complete.

## Practical Applications

The Plan and Execute pattern can be applied to various real-world scenarios:

- **Customer Support**: Helping customers troubleshoot complex issues step by step
- **Research Assistance**: Breaking down research questions into specific search queries and synthesis steps
- **Task Automation**: Creating workflows that combine multiple API calls and data transformations
- **Product Recommendations**: Gathering user preferences and matching them to suitable products

## Next Steps

To extend this pattern, consider:

1. Adding error handling to retry failed steps
2. Implementing dynamic replanning based on execution results
3. Incorporating user feedback between steps
4. Adding more specialized tools for specific domains

In [1]:
# Setup and Dependencies
import os
import json
import requests
import asyncio
import httpx
from urllib.parse import urljoin
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planners import SequentialPlanner
from semantic_kernel.planners.sequential_planner.sequential_planner_config import SequentialPlannerConfig
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from typing import List, Dict, Any
import nest_asyncio
from IPython.display import display, Markdown
import logging
import sys
sys.path.append(os.path.abspath('../../..'))  # Adjust the path as necessary
from utils.search_utils import web_search


# Apply nest_asyncio to allow nested event loops (required for Jupyter)
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Helper function to display markdown content
def display_md(text: str):
    display(Markdown(text))

In [2]:
# Load environment variables and configure Azure OpenAI
from dotenv import load_dotenv
load_dotenv()

# Get API keys from environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

# Check that environment variables are set
if not AZURE_OPENAI_ENDPOINT or not AZURE_OPENAI_API_KEY or not DEPLOYMENT_NAME:
    raise ValueError("Please set AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, and AZURE_OPENAI_DEPLOYMENT_NAME in .env file")
    
print(f"Azure OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Deployment Name: {DEPLOYMENT_NAME}")

# Check if Google Search API keys are available
if not GOOGLE_API_KEY or not GOOGLE_CSE_ID:
    print("Warning: GOOGLE_API_KEY or GOOGLE_CSE_ID not set. Web search functionality will be limited to demo mode.")

Azure OpenAI Endpoint: https://aoai-services1.openai.azure.com/
Deployment Name: gpt-4o-mini


In [3]:


async def extract_text_from_url(url):
    """
    Extract text content from a webpage URL.
    
    Args:
        url: The URL to extract text from
        
    Returns:
        Extracted text content
    """
    # For demo purposes, we'll generate some simulated content
    # In a real implementation, you would use httpx to fetch and parse the content
    
    # Simple simulation of webpage content based on URL
    if "example.com" in url:
        product_type = ""
        for product in ["smartphone", "laptop", "headphones", "smartwatch", "tablet"]:
            if product in url:
                product_type = product
                break
        
        if product_type:
            return f"This is a webpage about {product_type}s. It contains detailed information about features, specifications, and user reviews."
        else:
            return "This is a generic technology webpage with product information and reviews."
    
    return "Could not extract content from the provided URL."

async def add_context_async(urls):
    """
    Fetch and extract content from multiple URLs asynchronously.
    
    Args:
        urls: List of URLs to fetch content from
        
    Returns:
        List of extracted text content
    """
    tasks = [extract_text_from_url(url) for url in urls]
    results = await asyncio.gather(*tasks)
    return results

In [4]:
# Initialize Semantic Kernel with Azure OpenAI
kernel = sk.Kernel()

# Add Azure OpenAI service
kernel.add_service(
    AzureChatCompletion(
        service_id="azure_chat_completion",
        deployment_name=DEPLOYMENT_NAME,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY
    )
)

print("Semantic Kernel initialized with Azure OpenAI service")

Semantic Kernel initialized with Azure OpenAI service


## Creating Native Functions (Tools)

First, we'll create several native functions that our planner can use during execution. These functions represent the tools that our agent can use to complete tasks.

In [5]:
# Create a plugin with native functions that can be used by the planner
@kernel_function(
    description="Searches for information about a product.",
    name="search_product"
)
def search_product(query: str) -> str:
    """
    Searches for information about a specified product using web search.
    
    Args:
        query: The product to search for.
        
    Returns:
        Information about the product.
    """
    logger.info(f"Searching for product: {query}")
    
    # First try web search
    results = web_search(query, num=3)
    
    if results:
        # Process search results
        product_info = f"Found information about {query} from web search:\n\n"
        
        for i, result in enumerate(results):
            title = result.get("title", "Untitled")
            link = result.get("link", "")
            snippet = result.get("snippet", result.get("text_content", ""))
            
            product_info += f"{i+1}. {title}\n"
            product_info += f"   URL: {link}\n"
            if snippet:
                product_info += f"   Summary: {snippet}\n"
            product_info += "\n"
            
        return product_info
    
    return f"Could not find specific information about '{query}'. Available products in our local database are: {', '.join(static_product_database.keys())}."

@kernel_function(
    description="Compares features between two products.",
    name="compare_products"
)
def compare_products(product1: str, product2: str) -> str:
    """
    Compares features between two products using web search.
    
    Args:
        product1: First product to compare.
        product2: Second product to compare.
        
    Returns:
        Comparison between the products.
    """
    logger.info(f"Comparing products: {product1} vs {product2}")
    
    # Search for each product
    product1_results = web_search(product1, num=2)
    product2_results = web_search(product2, num=2)
    
    if product1_results and product2_results:
        comparison = f"Comparison between {product1} and {product2} based on web search:\n\n"
        
        # First product info
        comparison += f"## {product1.capitalize()} Information:\n"
        for i, result in enumerate(product1_results[:2]):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            comparison += f"{i+1}. {title}\n   {snippet}\n\n"
        
        # Second product info
        comparison += f"## {product2.capitalize()} Information:\n"
        for i, result in enumerate(product2_results[:2]):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            comparison += f"{i+1}. {title}\n   {snippet}\n\n"
        
        # Add comparison prompt
        comparison += f"Based on the information above, here are the key differences between {product1} and {product2}:"
        
        return comparison
    
    return f"No comparison data available between '{product1}' and '{product2}'. Try with more specific product types."

@kernel_function(
    description="Provides recommendations based on user preferences.",
    name="recommend_product"
)
def recommend_product(preferences: str) -> str:
    """
    Recommends products based on user preferences using web search.
    
    Args:
        preferences: User preferences for product recommendations.
        
    Returns:
        Product recommendations.
    """
    logger.info(f"Generating recommendations based on preferences: {preferences}")
    
    # Create a search query based on user preferences
    search_query = f"best products for {preferences}"
    
    # Perform web search
    results = web_search(search_query, num=3)
    
    if results:
        recommendations = f"Based on your preferences for '{preferences}', here are some recommended products:\n\n"
        
        for i, result in enumerate(results):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            
            recommendations += f"{i+1}. {title}\n"
            recommendations += f"   {snippet}\n\n"
            
        return recommendations
    
    return "Based on your preferences (from local database), I might need more specific information to make a tailored recommendation."

# Register functions with the kernel
# Register each function as a plugin
kernel.add_function(plugin_name="product_plugin", function=search_product)
kernel.add_function(plugin_name="product_plugin", function=compare_products)
kernel.add_function(plugin_name="product_plugin", function=recommend_product)

print("Native functions registered as tools")

Native functions registered as tools


## Creating Semantic Functions

Next, we'll create semantic functions that use natural language to perform tasks. These will be combined with our native functions in the planner.

In [6]:
# Create a semantic function for summarizing product information
summarize_prompt = """
You are a helpful product specialist.
Summarize the following product information in a concise and helpful manner.
Focus on the key features and benefits that would be most relevant to customers.

Product Information:
{{$input}}

Summary:
"""

summarize_function = kernel.add_function(
    function_name="summarize_product_info",
    plugin_name="product_plugin",
    prompt=summarize_prompt,
    description="Summarizes product information in a concise and helpful manner."
)

# Create a semantic function for generating product comparisons
compare_prompt = """
You are a helpful product specialist.
Create a detailed comparison between the products based on the information provided.
Include pros and cons for each product and mention which types of users would prefer each option.

Product Information:
{{$input}}

Comparison:
"""

compare_function = kernel.add_function(
    function_name="generate_detailed_comparison",
    plugin_name="product_plugin",
    prompt=compare_prompt,
    description="Generates a detailed comparison between products."
)

print("Semantic functions created")

Semantic functions created


## Setting Up the Planner

Now we'll configure the sequential planner, which will break down complex tasks into a sequence of steps using our available functions.

In [7]:
# Configure the sequential planner
planner_config = SequentialPlannerConfig(
    relevancy_threshold=0.7,
    max_tokens=1000,
    included_functions=["search_product", "compare_products", "recommend_product", "summarize_product_info", "generate_detailed_comparison"],
)

# Create the planner with the config as a named parameter
planner = SequentialPlanner(service_id="azure_chat_completion", kernel=kernel, config=planner_config)

print("Sequential planner configured")

Sequential planner configured


## Executing a Plan

Let's put our plan and execute pattern to work with a sample scenario. We'll ask the agent to help a customer find the right product based on their needs.

In [8]:
async def execute_plan_and_display(goal: str):
    
    
    
    """Execute a plan based on the specified goal and display the results."""
    print(f"🎯 Goal: {goal}")
    print("\n⚙️ Generating plan...\n")
    
    # Create and display the plan
    plan = await planner.create_plan(goal)
    plan_steps = "\n".join([f"Step {i+1}: {step.description}" for i, step in enumerate(plan.steps)])
    display_md(f"### Generated Plan\n{plan_steps}")
    
    print("\n🚀 Executing plan...\n")
    
    # Execute the plan
    result = await plan.invoke(kernel=kernel)
    print("\n✅ Plan execution completed\n")
    display_md(f"### Result\n{result}")

# Example goal
goal = "I need a recommendation for a product with good battery life for a college student."

# Execute the plan
await execute_plan_and_display(goal)

🎯 Goal: I need a recommendation for a product with good battery life for a college student.

⚙️ Generating plan...



### Generated Plan
Step 1: Provides recommendations based on user preferences.


🚀 Executing plan...

Using web search mode: bing

✅ Plan execution completed



### Result
Based on your preferences for 'good battery life for a college student', here are some recommended products:

1. Untitled
   No description available

2. Untitled
   No description available

3. Untitled
   No description available

4. Untitled
   No description available



## Try Different Goals

Let's try different types of user requests to see how our planner adapts.

In [9]:
# Let's try a comparison request
comparison_goal = "Compare the features of a samsung laptop and a samsung tablet for someone who needs to do both schoolwork and watch movies."

await execute_plan_and_display(comparison_goal)

🎯 Goal: Compare the features of a samsung laptop and a samsung tablet for someone who needs to do both schoolwork and watch movies.

⚙️ Generating plan...



### Generated Plan
Step 1: Searches for information about a product.
Step 2: Searches for information about a product.
Step 3: Generates a detailed comparison between products.


🚀 Executing plan...

Using web search mode: bing
Using web search mode: bing

✅ Plan execution completed



### Result
It appears that there may have been an error in the product information you intended to share, as there is no specific data or URLs provided for the Samsung tablet or any other competing product. However, I can create a hypothetical comparison between a Samsung tablet and another popular tablet brand, such as Apple’s iPad, as an example.

### Product Comparison: Samsung Tablet vs. Apple iPad

---

#### 1. Samsung Galaxy Tab S7+

**Overview:**
The Samsung Galaxy Tab S7+ is a high-end tablet characterized by its vibrant display, powerful processor, and versatility, especially when paired with the S Pen.

**Pros:**
- **Display:** 12.4-inch Super AMOLED display with a resolution of 2800 x 1752 pixels, offering vibrant colors and deep blacks.
- **Performance:** Powerful Snapdragon 865+ chipset, capable of handling tasks from graphic-intensive games to multitasking with multiple apps.
- **Stylus Support:** Comes with an S Pen included, perfect for note-taking, drawing, and other creative tasks.
- **Expandable Storage:** Supports microSD cards for additional storage, allowing users to easily expand their memory.
- **DeX Mode:** Offers a desktop-like experience when connected to a monitor, ideal for productivity users.

**Cons:**
- **Price:** More expensive than other tablets in the market, which might not fit all budgets.
- **Software Updates:** While Samsung provides updates, they may not be as timely as Apple’s iOS updates.
- **App Optimization:** Some apps may not be fully optimized for tablet use compared to their iOS counterparts.

**Ideal Users:**
- Creative professionals who benefit from the S Pen and large display.
- Multitaskers who need a reliable device for productivity.
- Users who prefer Android OS and want the flexibility of expandable storage.

---

#### 2. Apple iPad Pro (12.9-inch)

**Overview:**
The iPad Pro is Apple’s premium tablet, known for its exceptional build quality, strong performance, and a rich ecosystem of applications.

**Pros:**
- **Display:** 12.9-inch Liquid Retina display with ProMotion technology, resulting in ultra-smooth visuals and responsiveness.
- **Performance:** Equipped with the M1 chip, delivering high performance that can rival laptops.
- **App Ecosystem:** Access to a comprehensive range of apps, many of which are optimized for the iPad experience.
- **Apple Pencil Support:** The Apple Pencil (2nd generation) provides excellent precision for drawing and note-taking (sold separately).
- **Longevity and Resale Value:** iPads generally have longer software support and better resale values.

**Cons:**
- **Price:** Premium pricing can be a barrier for budget-conscious consumers.
- **No Expandable Storage:** Users are limited to the internal storage options, with no microSD support.
- **Accessories Extra Cost:** Accessories like the Apple Pencil and Magic Keyboard add to the overall price.

**Ideal Users:**
- Professionals and creatives who use design, illustration, and productivity software extensively.
- Apple ecosystem users who want seamless integration with their other Apple devices.
- Users looking for a device with high resale value and longevity.

---

### Conclusion

**Choosing the Right Tablet:**

- **Samsung Galaxy Tab S7+:** Ideal for users who enjoy flexibility with Android, want a stylus included for creative uses, and value expandable storage. Great for artists, students, and multitaskers.
  
- **Apple iPad Pro:** Best suited for users who are already invested in the Apple ecosystem, prioritize performance, and require apps that are well-optimized for the tablet interface. Excellent for professionals involved in creative fields or anyone needing a powerful productivity tool.

In summary, the decision largely boils down to personal preference in operating systems, desired features (like stylus functionality), and how much budget flexibility exists.

In [10]:
# Let's try a specific product search request
search_goal = "I want to know the camera quality and storage options for the smartphone."

await execute_plan_and_display(search_goal)

🎯 Goal: I want to know the camera quality and storage options for the smartphone.

⚙️ Generating plan...



### Generated Plan
Step 1: Searches for information about a product.
Step 2: Summarizes product information in a concise and helpful manner.
Step 3: Generates a detailed comparison between products.


🚀 Executing plan...

Using web search mode: bing

✅ Plan execution completed



### Result
It seems there might have been some misunderstanding as no specific products or their details were provided for comparison. However, I can help you create a structured outline for comparing smartphones based on hypothetical or commonly found features. If you have specific smartphones in mind or can provide more details, feel free to share, and I will tailor the comparison accordingly.

Here's a structured comparison format you can use:

### Smartphone A vs. Smartphone B

#### Key Features
- **Smartphone A**:
  - Display Size: 6.5 inches, AMOLED
  - Processor: Snapdragon 888
  - Camera: Triple-lens (108MP main, 12MP ultrawide, 5MP macro)
  - Battery: 4500mAh with fast charging
  - OS: Android 12
  - Storage Options: 128GB, 256GB with expandable storage

- **Smartphone B**:
  - Display Size: 6.2 inches, LCD
  - Processor: MediaTek Dimensity 1200
  - Camera: Dual-lens (64MP main, 12MP ultrawide)
  - Battery: 5000mAh with standard charging
  - OS: Android 11
  - Storage Options: 64GB, 128GB without expandable storage

#### Pros and Cons

**Smartphone A:**
- **Pros:**
  - High-resolution camera ideal for photography enthusiasts.
  - AMOLED display offering better colors and contrast.
  - Faster processor, great for gaming and multitasking.
  - Expandable storage helps accommodate more files and apps.
  
- **Cons:**
  - Smaller battery compared to Smartphone B, potentially leading to shorter usage time.
  - Premium features might lead to a higher price point.

- **Target Users:**
  - Ideal for tech-savvy users who prioritize camera quality, gaming performance, and media consumption.

**Smartphone B:**
- **Pros:**
  - Larger battery capacity ensures longer usage on a full charge.
  - More affordable price point, making it budget-friendly.
  - Decent performance for everyday tasks and moderate gaming.

- **Cons:**
  - LCD display may not provide vibrant colors compared to AMOLED.
  - Limited camera setup may not appeal to serious photography enthusiasts.
  - No option for expandable storage can limit file storage options.

- **Target Users:**
  - Suitable for budget-conscious users who need a reliable smartphone for daily use without advanced photography or gaming features.

### Conclusion
In choosing between Smartphone A and Smartphone B, potential users should consider their priorities. Those seeking high performance, excellent camera capabilities, and vibrant displays might lean towards Smartphone A, while those looking for a more budget-friendly, reliable, and longer-lasting device might find Smartphone B to be more suitable.

If you can provide specific smartphone models or additional details, I'll create a more tailored and accurate comparison!